---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def plot_loglog(G):
    #G = P1_Graphs[4]
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

    plt.plot(degree_values, histogram, 'o')
    plt.xlabel('Degree')
    plt.ylabel('Fraction of Nodes')
    plt.xscale('log')
    plt.yscale('log')
    plt.show()

In [25]:
def plot_bar(G):
    
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]

    plt.bar(degree_values, histogram)
    plt.xlabel('Degree')
    plt.ylabel('Fraction of Nodes')
    plt.show()
    
    print('Avg. Clustering Coeff:', nx.average_clustering(G))
    print('Avg. Shortest Path:', nx.average_shortest_path_length(G))
    
    return

In [5]:
def graph_identification(): 
    
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [6]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [7]:
def question_two_data():
    from sklearn.model_selection import train_test_split
    
    # Initialize the dataframe, using the nodes as the index
    salary_df = pd.DataFrame(index=G.nodes())
    # Extact node attributes and add to DF
    salary_df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    salary_df['management_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    # Compute node measurements and add to DF
    salary_df['clustering'] = pd.Series(nx.clustering(G))
    salary_df['degree'] = pd.Series(nx.degree(G))
    #Split DF into train and predict DF
    predict_df = salary_df[salary_df['management_salary'].isnull()]
    train_df = salary_df[~(salary_df['management_salary'].isnull())]
    # Split train_df into X and y
    X = train_df[['department', 'clustering', 'degree']]
    y = train_df['management_salary']
    
    return (X, y, predict_df)

In [8]:
def salary_predictions():
    from sklearn.linear_model import LogisticRegression
    # Gets preprocessing data
    X, y, predict_df = question_two_data()    
    # Trains model and predicts on new data
    clf = LogisticRegression(penalty='l2', C=10)
    predict_prob = clf.fit(X, y).predict_proba(predict_df[['department', 'clustering', 'degree']])    
    predict =  predict_prob[:,[1]]
    
    return pd.Series(predict[:,0], index = predict_df.index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [18]:
#future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
#future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
def build_graph(future_connections):
    
    # Create a set of nodes in dataframe
    node_labels = future_connections.index
    node_labels = list(zip(*node_labels))
    node_labels = set([node_labels[0]+node_labels[1]][0])
    # Get all edges in dataframe
    edges = future_connections[future_connections['Future Connection'] == 1].index
    # Make graph and add nodes and edges
    future_connections_G = nx.Graph()
    future_connections_G.add_nodes_from(node_labels)
    future_connections_G.add_edges_from(edges)
    
    return future_connections_G

In [11]:
def link_prediction(dataframe, generator, prediction_name):
    
    # Int list to add generator values
    indexies = list()
    scores = list()
    # Loop over generator and store values
    for u, v, p in generator:
        indexies.append((u ,v))
        scores.append(p)
    # Add scores to dataframe with prediction_name
    dataframe[prediction_name] = scores
    
    return dataframe

In [12]:
def cal_common_neighbors(future_connections, future_connections_G):
    # Int list to add generator values
    indexies = list()
    scores = list()
    # Loop over node pairs and calculate score
    for u, v in future_connections.index:
        p  = nx.common_neighbors(future_connections_G, u, v)
        indexies.append((u ,v))
        scores.append(len(sorted(p)))
    # Add score to dataframe
    future_connections['common_neighbors'] = scores
    
    return future_connections

In [13]:
def pre_preprocessing(future_connections, future_connections_G):
    # Calculate Resource Allocation Index
    future_connections = link_prediction(future_connections, 
                                     nx.resource_allocation_index(future_connections_G, future_connections.index),
                                     'resource_allocation_index')
    # Calculate Jaccard Coefficents
    future_connections = link_prediction(future_connections, 
                                     nx.jaccard_coefficient(future_connections_G, future_connections.index),
                                     'jaccard_coefficient')
    # Calculate Adamic Adar
    future_connections = link_prediction(future_connections, 
                                     nx.adamic_adar_index(future_connections_G, future_connections.index),
                                     'adamic_adar_index')
    # Calculate Preferential Attachment
    future_connections = link_prediction(future_connections, 
                                     nx.preferential_attachment(future_connections_G, future_connections.index),
                                     'preferential_attachment')
    # Calculate Common Neighbors
    future_connections = cal_common_neighbors(future_connections, future_connections_G)
    
    return future_connections

In [14]:
def split_data(future_connections):
    
    predict_df = future_connections[future_connections['Future Connection'].isnull()]
    train_df = future_connections[~(future_connections['Future Connection'].isnull())]
    
    y = train_df.iloc[:, 0]
    X = train_df.iloc[:, 1:]
    
    return (predict_df, X, y)

In [15]:
def preform_grid_search():
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    # Split data into training and testing set
    predict_df, X, y = split_data(future_connections)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    # Perform Logistics Regression Grid Search
    clf = LogisticRegression()
    grid_values = [{'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10, 100]}]
    grid_clf_rec = GridSearchCV(clf, param_grid = grid_values, scoring ='roc_auc')
    grid_clf_rec.fit(X_train, y_train)
    y_decision_fn_scores_rec = grid_clf_rec.decision_function(X_test)
    
    return grid_clf_rec.cv_results_['mean_test_score'].reshape(5,2)

In [16]:
def new_connections_predictions():
    from sklearn.linear_model import LogisticRegression
    
    # Gets raw data
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    # Gets preprocessing data
    future_connections_G = build_graph(future_connections)
    future_connections = pre_preprocessing(future_connections, future_connections_G)
    # Splits datasets    
    predict_df, X, y = split_data(future_connections)   
    # Trains model and predicts on new data
    clf = LogisticRegression(penalty='l1', C=0.01)
    predict_prob = clf.fit(X, y).predict_proba(predict_df.iloc[:, 1:])    
    predict =  predict_prob[:,[1]]
    
    return pd.Series(predict[:,0], index = predict_df.index)